## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import google_api_key

# Configure gmaps API key
gmaps.configure(api_key=google_api_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")

In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum desired temp (in F)? "))
max_temp = float(input("What is the maximum desired temp (in F)? "))

What is the minimum desired temp (in F)? 60
What is the maximum desired temp (in F)? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
cities_within_temps = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp), :]

In [6]:
# 4a. Determine if there are any empty rows.
cities_within_temps.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = cities_within_temps.dropna()

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Camacha,PT,63.82,broken clouds,33.0833,-16.3333,
2,Arraial Do Cabo,BR,67.68,overcast clouds,-22.9661,-42.0278,
6,Jamestown,US,64.94,overcast clouds,42.0970,-79.2353,
11,Vila Franca Do Campo,PT,64.20,few clouds,37.7167,-25.4333,
14,Natal,BR,72.25,few clouds,-5.7950,-35.2094,
15,Bestobe,KZ,61.48,overcast clouds,52.4994,73.0975,
18,Praia Da Vitoria,PT,61.05,few clouds,38.7333,-27.0667,
19,Port Alfred,ZA,69.87,overcast clouds,-33.5906,26.8910,
20,Trapani,IT,69.89,clear sky,37.8333,12.6667,
22,Novopokrovskaya,RU,69.01,overcast clouds,45.9514,40.7006,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": google_api_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotel_data = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
    except:
        pass

In [10]:
# 7. Drop the rows where there is no Hotel Name.
#hotel_df.isnull().sum()
clean_hotel_df = hotel_df.replace(r'^s*$', float('NaN'), regex=True)
clean_hotel_df.dropna(inplace=True)
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Camacha,PT,63.82,broken clouds,33.0833,-16.3333,Hotel Porto Santo & Spa
2,Arraial Do Cabo,BR,67.68,overcast clouds,-22.9661,-42.0278,Pousada Porto Praia
6,Jamestown,US,64.94,overcast clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
11,Vila Franca Do Campo,PT,64.20,few clouds,37.7167,-25.4333,Pestana Bahia Praia
14,Natal,BR,72.25,few clouds,-5.7950,-35.2094,Villa Park Hotel | Natal
...,...,...,...,...,...,...,...
699,Shetpe,KZ,73.08,overcast clouds,44.1667,52.1167,Rtsm Shetpe Analog
704,Benguela,AO,68.38,clear sky,-12.5763,13.4055,Hotel Praia Morena
707,Camana,PE,61.03,overcast clouds,-16.6228,-72.7111,Hostal Morales
708,Gilgit,PK,74.73,few clouds,35.9221,74.3087,Tourist Hamlet Gilgit


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "./WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# Create map
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))